# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,okhotsk,59.3833,143.3000,-19.81,76,4,5.24,RU,1736435369
1,1,ciudad lazaro cardenas,17.9583,-102.2000,24.90,66,100,1.50,MX,1736435115
2,2,albany,42.6001,-73.9662,-6.97,67,67,4.47,US,1736435359
3,3,stanley,54.8680,-1.6985,0.29,77,20,5.66,GB,1736435372
4,4,alofi,-19.0595,-169.9187,23.94,94,75,1.54,NU,1736435373


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
# Configure the map plot
city_map = city_data_df.hvplot.points(
    x="Lng",              
    y="Lat",               
    geo=True,             
    tiles="CartoLight",  
    size='Humidity', 
    color='City',
    frame_width=700,       
    frame_height=500,      
    xlim=(-180, 180),      
    ylim=(-90, 90),        
    hover_cols=["City"],   
)


# Display the map
city_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

<Figure size 640x480 with 0 Axes>

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_df = city_data_df[
    (city_data_df['Max Temp'] >= 21) & 
    (city_data_df['Max Temp'] <= 29) &
    (city_data_df['Humidity'] >= 40) & 
    (city_data_df['Humidity'] <= 80) &
    (city_data_df['Cloudiness'] < 20)
    
]

# Drop any rows with null values
filtered_df = filtered_df.dropna()

# Display sample data
filtered_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,hermanus,-34.4187,19.2345,25.45,63,0,9.70,ZA,1736435411
42,42,antofagasta,-23.6500,-70.4000,21.60,70,2,4.26,CL,1736435226
51,51,andergrove,-21.0833,149.1833,24.87,78,0,2.57,AU,1736435429
59,59,salalah,17.0151,54.0924,24.05,60,0,1.03,OM,1736435438
60,60,pedernales,18.0384,-71.7440,26.74,71,5,2.97,DO,1736435439


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column "Hotel Name" to the DataFrame
hotel_df['Hotel Name'] = None

# Display sample data
hotel_df.head()



,City,Country,Lat,Lng,Humidity,Hotel Name
36,hermanus,ZA,-34.4187,19.2345,63,None
42,antofagasta,CL,-23.6500,-70.4000,70,None
51,andergrove,AU,-21.0833,149.1833,78,None
59,salalah,OM,17.0151,54.0924,60,None
60,pedernales,DO,18.0384,-71.7440,71,None


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "radius": radius,
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    
    # get latitude, longitude from the DataFrame
    city_lat = row["Lat"]
    city_lng = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{city_lng},{city_lat},{radius}"  
    params["bias"] = f"proximity:{city_lng},{city_lat}"  

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_df = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        features = name_address_df["features"]
        if features:
            hotel_name = features[0]["properties"]["name"]
            hotel_df.loc[index, "Hotel Name"] = hotel_name
        else:
            hotel_df.loc[index, "Hotel Name"] = "No hotel found"
    except (KeyError, IndexError):
        hotel_df.loc[index, "Hotel Name"] = "Error retrieving hotel"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

  

Starting hotel search
hermanus - nearest hotel: Aloe guest house
antofagasta - nearest hotel: Hotel Colón
andergrove - nearest hotel: Error retrieving hotel
salalah - nearest hotel: Muscat International Hotel
pedernales - nearest hotel: hotel Isla beata
port alfred - nearest hotel: Error retrieving hotel
bredasdorp - nearest hotel: Victoria Hotel
carnarvon - nearest hotel: No hotel found
nova sintra - nearest hotel: Residência Ka Dencho
port saint john's - nearest hotel: Outback Inn
mejit - nearest hotel: No hotel found
syriam - nearest hotel: Excel River View
port elizabeth - nearest hotel: Waterford Hotel
north ward - nearest hotel: Aquarius on the Beach
kailua-kona - nearest hotel: PACIFIC 19 Kona
pisco - nearest hotel: La Portada
east london - nearest hotel: Error retrieving hotel
sokoni - nearest hotel: Error retrieving hotel
al ghayzah - nearest hotel: فندق تاج العرب
moramanga - nearest hotel: Hotel Vanilla
bogale - nearest hotel: No hotel found
mangrol - nearest hotel: No hotel 

,City,Country,Lat,Lng,Humidity,Hotel Name
36,hermanus,ZA,-34.4187,19.2345,63,Aloe guest house
42,antofagasta,CL,-23.6500,-70.4000,70,Hotel Colón
51,andergrove,AU,-21.0833,149.1833,78,Error retrieving hotel
59,salalah,OM,17.0151,54.0924,60,Muscat International Hotel
60,pedernales,DO,18.0384,-71.7440,71,hotel Isla beata


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:


city_map = hotel_df.hvplot.points(
    x="Lng",              
    y="Lat",               
    geo=True,              
    tiles="OSM",          
    color="City",         
    hover_cols=["City", "Country", "Humidity", "Hotel Name"],  
    frame_width=700,      
    frame_height=500,      
    xlim=(-180, 180),      
    ylim=(-90, 90)        
)

# Display the map
city_map

# Display the map
# YOUR CODE HERE

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Humidity,Hotel Name)